In [1]:
import wandb
import numpy as np
import random

from itertools import product
from wandb.keras import WandbCallback
from numba import jit

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, Concatenate
from tensorflow.keras.losses import CategoricalCrossentropy, Huber, MeanSquaredError
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from tf_agents.environments import py_environment
from tf_agents.trajectories import trajectory
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import array_spec, tensor_spec

from spektral.layers import XENetDenseConv
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.data import Graph
from spektral.data.dataset import Dataset
from spektral.data.loaders import BatchLoader

2022-06-06 07:41:31.288586: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-06 07:41:31.288637: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# initialize wandb

wandb.init(project="Test - D=2 & L=3", 
           entity="locp")

wandb: Currently logged in as: filippo_festa (locp). Use `wandb login --relogin` to force relogin
2022-06-06 07:41:36.376471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-06 07:41:36.376526: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.9/site-packages/wandb/sdk/lib/ipython.py:47: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML  # type: ignore


## Predefined Functions 

### Adjacency Matrix 

In [3]:
def Adj(D, L, sparse=False):
    N = L**D

    # create all nodes' coordinates
    nodes = [x for x in np.ndindex(tuple(np.repeat(L,D)))]

    # Pass from coordinate to node's index
    # (h,...k,j,i) <=> index = h*L^(D-1) + ... + k*L^2 + j*L + i
    mul = [L**i for i in reversed(range(D))]

    # Creation of adjacency matrix 
    A_dense = []
    # creation of a row for each node's coordinate 
    for node in nodes:       
        temp_buffer = []
        A_dense_row = [0]*N
        # find the two nearest neighbours of the node along each dimension
        for d in range(D):
            temp=list(node)
            temp[d]=((temp[d]+1)%L)
            temp=np.inner(temp, mul)
            temp_buffer.append(temp)    

            temp=list(node)
            temp[d]=((temp[d]-1)%L)
            temp=np.inner(temp, mul)
            temp_buffer.append(temp)
      
        temp_buffer=list(np.unique(np.array(temp_buffer), axis=0))   
        for i in temp_buffer: A_dense_row[i]=1
        A_dense.append(A_dense_row)
    
    # sparse=False => sparse adjacency matrix
    # sparse=True => dense adjacency matrix
    if sparse:
        return sp_matrix_to_sp_tensor(np.array(A_dense))
    else:
        return np.array(A_dense)

### Interaction Matrix 

In [4]:
def J_inter(denseAdj):
    N = denseAdj.shape[0]
    sparseAdj = sp_matrix_to_sp_tensor(denseAdj)

    # sparse adjacency matrix as a numpy array
    edge=sparseAdj.indices.numpy()

    # ordered numpy sparse adjacency matrix
    un_edge=np.array([np.sort(i) for i in edge]) 

    # creation of the interaction array: (i,j) and (j,i) have the same Jij
    inter=[]
    for i in range(len(un_edge)):
        equal=True
        for j in range(i):
            if np.array_equal(un_edge[i],un_edge[j]):
                inter.append(inter[j])
                equal=False
                break
        if equal: 
            inter.append(np.random.normal(0, 1))
    
    # creation of dense interaction matrix
    inter_matrix = np.zeros((N,N))
    counter = 0
    for i, j in edge:
        inter_matrix[i,j] = inter[counter]
        counter += 1
    return [np.array(inter).reshape(sparseAdj.indices.shape[0],1), inter_matrix.reshape((N,N,1))]
    
    # index of the returned list:
    # 0 => interaction array
    # 1 => interaction matrix (zero padded)

### Dataset

In [5]:
class MyDataset(Dataset):
    def __init__(self, N_graph, X, Y, A, E, **kwargs):
        self.X = X
        self.Y = Y
        self.N_graph = N_graph
        self.A = A
        self.E = E
        super().__init__(**kwargs)

    def read(self):
        mydataset = []
        for i in range(self.N_graph):
            # list of Graph objects that will be used as input in the BatchLoader
            mydataset.append(
                    Graph(x=self.X[i], a=self.A[i], e=self.E[i], y=self.Y[i])      
                    )
        return mydataset

### Replay Memory Buffer

In [6]:
# save n-step transitions (s_t; a_t; r_t,t+n; s_t+n) from the trajectory buffer

def get_replay_memory(trajectory_buffer, replay_memory):
    n = len(trajectory_buffer)

    states = np.array([transition[0] for transition in trajectory_buffer])
    actions = np.array([transition[1] for transition in trajectory_buffer])
    rewards = np.array([transition[2] for transition in trajectory_buffer])
    done = np.array([transition[4] for transition in trajectory_buffer])
    inter_matrix = trajectory_buffer[0][-1]
    dense_AdjMat = trajectory_buffer[0][-2]

    cum_reward = np.cumsum(rewards)

    # creating the replay memory buffer from the trajectory one
    # => (starting state, action performed, cumulative reward after n step from the starting one, state after n step from the starting one, episode ended, dense adjacency matrix of the episode, interaction matrix of the episode)
    replay_memory.append([states[0], actions[0], cum_reward[n-1], states[n-1], done[n-1], dense_AdjMat, inter_matrix])

    return replay_memory

### Real Energy Minima

In [7]:
@jit(nopython=True)
def computeEnergy(state, edge, interaction):  
    energy = 0
    for i in range(len(edge)):
        energy -= interaction[i][0]*state[edge[i][0]][0]*state[edge[i][1]][0]
    return energy/2


@jit(nopython=True)
def EnergyMinima(states, inter, sparseAdj):
    # find the minimum energy for a fixed J_ij configuration
    energy_min = np.inf
    for state in states:
        state = np.array(state).reshape(N,1)
        state_energy = computeEnergy(state, sparseAdj, inter)
        if state_energy<energy_min: energy_min=state_energy
    return energy_min

## Environment 

In [8]:
class SG_env(py_environment.PyEnvironment):

  def __init__(self, L, D):
    # Initialize environment attributes
    # - action_spec: action declaration (integer from 0 to N-1)
    # - observation_spec: state of the system declaration
    # - episode_ended: flag for the end of the episode (all spin down)
    # - state: state of the system (1 and -1 array)
    self.N = L**D
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=self.N-1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(self.N,1), dtype=np.int32, minimum=-1, maximum=1, name='observation')
    self.sp_AdjMat = Adj(D, L, sparse=True)
    self.dense_AdjMat = Adj(D, L, sparse=False)
    self.interaction = J_inter(self.dense_AdjMat)[0]   
    self.inter_matrix = J_inter(self.dense_AdjMat)[1]
    self._state = np.ones(shape=(self.N,1)).astype("int32")
    self._episode_ended = False

  def get_state(self):
    return self._state

  def show_N(self):
    return self.N  

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def show_dense_AdjMat(self):
    return self.dense_AdjMat

  def show_sp_AdjMat(self):
    return self.sp_AdjMat

  def show_interaction(self):
    return self.interaction

  def show_inter_matrix(self):
    return self.inter_matrix

  # True => All spins = -1, False => otherwise
  def __all_spins_down(self):
    return np.all(self._state==-1)    

  # Compute the reward of the chosen action 
  # reward = energy difference between consecutive states
  # nns => nearest neighbours indexes
  # nn_Js => nearest neighbours interactions' indexes
  def computeReward(self, action):
    nns = self.sp_AdjMat.indices[self.sp_AdjMat.indices[:,0]==action][:,1].numpy()
    nn_Js = np.where(self.sp_AdjMat.indices[:,0]==action)[0]
    nn_sum = 0
    for i in range(len(nns)): nn_sum += self.interaction[nn_Js[i]]*self._state[nns[i],0]
    reward = 2*nn_sum*self._state[action,0]
    return reward[0]

  # Compute the energy of the current state 
  def computeEnergy(self):
    edge = self.sp_AdjMat.indices.numpy()
    Nedge = len(edge)
    energy = 0
    for i in range(Nedge):
        energy -= self.interaction[i][0]*self._state[edge[i][0]][0]*self._state[edge[i][1]][0]
    return energy/2

  # reset function: called when all spins are -1 => new episode
  #                                              => all spins up (=1) and new interaction matrix (if needed)
  def _reset(self):
    self._state = np.ones(shape=(self.N,1)).astype("int32")
    #self.interaction = J_inter(self.dense_AdjMat)[0]    
    #self.inter_matrix = J_inter(self.dense_AdjMat)[1]
    self._episode_ended = False
    return ts.restart(np.array(self._state, dtype=np.int32))

  # step function: describe the process of applying the action selected by the agent
  # ts.restart, ts.transition and ts.termination return a timestep 
  # containing step_type, reward, discount and observation
  def _step(self, action):
    if self._episode_ended:
      return self.reset()

    if self.__all_spins_down():
      self._episode_ended = True
    elif (action>=0 and action<=self.N-1) and (self._state[action,0]==1):
      self._state[action,0]=-1
      rew = self.computeReward(action)
      
      if self.__all_spins_down():
          self._episode_ended = True
          return ts.termination(np.array(self._state, dtype=np.int32), reward=rew)
      else:
          return ts.transition(np.array(self._state, dtype=np.int32), reward=rew)
    
    elif (action>=0 and action<=self.N-1) and (self._state[action,0]==-1):
      raise ValueError('Each spin can be flipped only once!')
    else:
      raise ValueError('`action` should be 0 up to N-1 - Spin Flip!')

## Agent

In [9]:
# Agent (=> GNN+FNN) 
# N => number nodes
# D => number dimensions
# stack_channels => integer or list of integers, number of channels for the hidden layers
# node_channels => integer, number of output channels for the nodes
# edge_channels => integer, number of output channels for the edges
# division_factor_dense => integer, gradually reduce the number of neurons for each dense layer
# p_drop => float between 0 and 1, fraction of the input units to drop
# N_xenet => Number of XENetDenseConv layers
# N_dense => Number of Dense hidden layers

def agent(N,                                 
          D,                                 
          stack_channels=5,        
          node_channels=3,
          edge_channels=3,
          division_factor_dense=4,
          p_drop=0,
          N_xenet=2,
          N_dense=2,
          activation="relu",
          regularizer=0):
  inX = Input(shape=(N,1), name='Input Nodes')
  inA = Input(shape=(N,N), name='Input Adj matrix')
  inE = Input(shape=(N,N,1), name='Input Edges')
  
  X, E =  XENetDenseConv(stack_channels, node_channels, edge_channels,
                               attention=True, node_activation=activation, edge_activation=activation, 
                               kernel_regularizer=l2(regularizer), name="XENet_layer_0")([inX, inA, inE])
  for i in range(N_xenet-1):
    X, E =  XENetDenseConv(stack_channels, node_channels, edge_channels,
                               attention=True, node_activation=activation, edge_activation=activation, 
                               kernel_regularizer=l2(regularizer), name="XENet_layer_"+str(i+1))([X, inA, E])
  
  # flat the updated X, E in order to feed the fully connected neural network (FNN)
  flat_x, flat_e = Flatten(name="Nodes_encoding")(X), Flatten(name="Edges_encoding")(E)
  out = Concatenate(axis=1, name="Concatenation")([flat_x])    #,flat_e
  
  for i in range(N_dense):
    out = Dense(out.shape.as_list()[1]//division_factor_dense, activation=activation, kernel_regularizer=l2(regularizer))(out)
    out = Dropout(p_drop)(out)
  out = Dense(N, activation="PReLU", kernel_regularizer=l2(regularizer), name='Q-values')(out)
  
  model = Model([inX,inA,inE], out)
  model.compile(optimizer=Adam(), loss=MeanSquaredError())
  return model

## Training


In [10]:
def train(env, replay_memory, model, target_model, done):
    discount_factor = 0.618
    
    # skip the training if the number of samples in the replay 
    # memory is less than MIN_REPLAY_SIZE
    MIN_REPLAY_SIZE = 100
    if len(replay_memory) < MIN_REPLAY_SIZE:
        return

    # randomly select a number of samples from the replay memory equal to batch_size
    batch_size = 60
    mini_batch = random.sample(replay_memory, batch_size)

    # sets of all interaction matrix and dense adjacency matrix in the batch
    E = np.array([transition[-1] for transition in mini_batch])
    A = np.array([transition[-2] for transition in mini_batch])

    # current_states => set of all starting observations in the batch
    # current_qs_list => predicted Q-values of the current_states by the model
    current_states = np.array([transition[0] for transition in mini_batch])
    current_qs_list = np.array(model.predict([current_states,A,E]))
 
    # new_current_states => set of observations after performing n actions in the batch
    # future_qs_list => predicted Q-values of the new_current_states by the target model
    new_current_states = np.array([transition[3] for transition in mini_batch])
    future_qs_list = np.array(target_model.predict([new_current_states,A,E]))
    
    # X => observations
    # Y => 'label' of each observation: updated current_qs_list 
    X = []
    Y = []
    for index, (observation, action, reward, new_observation, done, dense_AdjMat, inter_matrix) in enumerate(mini_batch):
        if not done:
            new_q = reward + discount_factor*np.max(future_qs_list[index])
        else:
            new_q = reward

        # update the Q-value corrisponding to the performed action
        current_qs = current_qs_list[index]
        current_qs[action] = new_q
        
        X.append(observation)
        Y.append(current_qs)

    # build the training dataset 
    train_data = MyDataset(N_graph=batch_size, X=X, Y=Y, A=A, E=E)
    # use the BatchLoader to fit the model and WandbCallback to load the loss to Weight&Biases
    loader = BatchLoader(train_data, node_level=False, epochs=50, batch_size=batch_size, shuffle=False) 
    model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, verbose=2,
                callbacks = [WandbCallback()])

In [11]:
def main(env): 
    
    # Epsilon-greedy algorithm in initialized at 1 
    # => every step is random at the start
    epsilon = 1          
    max_epsilon = 1     
    min_epsilon = 0.01  
    decay = 0.01

    # 1. Initialize the Target and Main models 
    # Main Model (updated every 3 steps)
    model = agent(N=env.N, D=D, stack_channels=73, node_channels=20, edge_channels=20,
                  N_xenet=1, N_dense=17, division_factor_dense=1)

    # Target Model (updated at the end of every episode)
    target_model = agent(N=env.N, D=D, stack_channels=73, node_channels=20, edge_channels=20,
                         N_xenet=1, N_dense=17, division_factor_dense=1)
    target_model.set_weights(model.get_weights())

    energy_buffer = []
    replay_memory = []
    
    step_buffer = 8
    set_target = 9
    train_episodes = 300
    for episode in range(train_episodes):
        # reset the variables at the beginning of an episode
        trajectory_buffer = []   
        ep_min_energy = np.inf
        steps_to_update_target_model = 0
        env.reset()
        previous_obs = np.ones(shape=(env.N,1)).astype("int32")
        done = False  
        check = np.arange(0,env.N)

        while not done: 
            observation = env.get_state()          
            print("\n\n\t\t\t\t++++++++++++  episode:", episode," - step:", steps_to_update_target_model, " ++++++++++++")

            # 2. Explore using the Epsilon Greedy Exploration Strategy
            random_number = np.random.rand()
            if random_number <= epsilon:
                # Explore
                action = random.choice(check)

            else:
                # Exploit best known action
                predicted = model([observation.reshape(1,env.N,1), env.dense_AdjMat.reshape(1,env.N,env.N), env.inter_matrix.reshape(1,env.N,env.N,1)], training=False).numpy()[0]
                while True:
                    # check to prevent flipping the same spin twice - only once!
                    action = np.argmax(predicted)
                    if env.get_state()[action,0] == 1:
                            break;
                    predicted[action] = np.NINF

            # remove the choosen action from check array
            check = np.setdiff1d(check, action)

            # perform the action on the environment and get the updated parameters
            step_type, reward, discount, new_observation = env._step(action)
            done = env._episode_ended
            e = env.computeEnergy()

            # save the parameter in the buffer
            trajectory_buffer.append([previous_obs, action, reward, new_observation, done, e, env.dense_AdjMat, env.inter_matrix])
            energy_buffer.append([episode, new_observation, env.interaction, e])  
            if ep_min_energy>e: ep_min_energy = e
            
            # load interesting parameters to weight&Biases
            wandb.log({
                "Episode": energy_buffer[episode*env.N+steps_to_update_target_model][0],
                "Step": episode*env.N+steps_to_update_target_model,
                "New observation": wandb.Image(energy_buffer[episode*env.N+steps_to_update_target_model][1].reshape(L,L)),
                "J interactions": energy_buffer[episode*env.N+steps_to_update_target_model][2],
                "Energy": energy_buffer[episode*env.N+steps_to_update_target_model][3]
            })

            # fill the replay memory buffer
            if steps_to_update_target_model >= step_buffer:   
                replay_memory = get_replay_memory(trajectory_buffer, replay_memory)
                trajectory_buffer = trajectory_buffer[1:]

            # 3. Update the Main Network using the Bellman Equation  
            #if (steps_to_update_target_model%L==0 and steps_to_update_target_model!=0) or done:
            print("\n\t\t\t\t\t      +++++ Training +++++")
            train(env, replay_memory, model, target_model, done)
  
            previous_obs = new_observation

            # Copying main network weights to the target network
            # weights at the end of the episode
            if done:
                if episode >= set_target:
                    target_model.set_weights(model.get_weights())
                break

            steps_to_update_target_model += 1

        # update epsilon using the following rule
        epsilon = min_epsilon+ (max_epsilon -min_epsilon) * np.exp(-decay *episode)
        wandb.log({"Episode Energy Minima" : ep_min_energy})

    return target_model

## Testing

In [12]:
def test(env, model, L): 
    env.reset()
    energy = []
    done = False

    while not done: 
        observation = env.get_state()          
        wandb.log({"Test state": wandb.Image(observation.reshape(L,L))})
        
        # Exploit best known action
        predicted = model([observation.reshape(1,env.N,1), env.dense_AdjMat.reshape(1,env.N,env.N), env.inter_matrix.reshape(1,env.N,env.N,1)], training=False).numpy()[0]
        while True:
            #check to prevent flipping the same spin twice - only once!
            action = np.argmax(predicted)
            if env.get_state()[action,0] == 1:
                    break;
            predicted[action] = np.NINF

        step_type, reward, discount, new_observation = env._step(action) 
        done = env._episode_ended
        e = env.computeEnergy()
        energy.append([new_observation, e])
        
    ground_state = np.min([e[1] for e in energy])         
    return ground_state

# GO!

In [13]:
L = 3
D = 2
N = L**D   
environment = SG_env(L=L, D=D)

# train and save the model
trained_targetModel = main(environment)

2022-06-06 07:41:38.959415: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-06 07:41:38.959472: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-06 07:41:38.959492: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (12846803c17b): /proc/driver/nvidia/version does not exist
2022-06-06 07:41:38.960047: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.




				++++++++++++  episode: 0  - step: 0  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 0  - step: 1  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 0  - step: 2  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 0  - step: 3  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 0  - step: 4  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 0  - step: 5  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 0  - step: 6  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 0  - step: 7  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 0  - step: 8  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 1  - step: 0  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 1  - step: 1  ++++++++++++

					      +++++ Training +++++


				++++++++++++  e

/opt/conda/lib/python3.9/site-packages/wandb/sdk/data_types/image.py:439: RuntimeWarning: invalid value encountered in true_divide
  data = (data - np.min(data)) / np.ptp(data)



					      +++++ Training +++++


				++++++++++++  episode: 2  - step: 5  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 2  - step: 6  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 2  - step: 7  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 2  - step: 8  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 3  - step: 0  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 3  - step: 1  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 3  - step: 2  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 3  - step: 3  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 3  - step: 4  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 3  - step: 5  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 3  - step: 6  ++++++++++++

					      +++++ Tra



				++++++++++++  episode: 13  - step: 0  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 13  - step: 1  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 13  - step: 2  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 13  - step: 3  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 13  - step: 4  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 13  - step: 5  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 13  - step: 6  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 13  - step: 7  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 13  - step: 8  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 14  - step: 0  ++++++++++++

					      +++++ Training +++++


				++++++++++++  episode: 14  - step: 1  ++++++++++++

					      +++++ Training +++++


				++++

...



				++++++++++++  episode: 279  - step: 1  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 6.1855e-15 - _timestamp: 1654501725.0000 - _runtime: 431.0000 - 41ms/epoch - 41ms/step


				++++++++++++  episode: 279  - step: 2  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 1.0972e-14 - _timestamp: 1654501726.0000 - _runtime: 432.0000 - 45ms/epoch - 45ms/step


				++++++++++++  episode: 279  - step: 3  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 1.2737e-14 - _timestamp: 1654501726.0000 - _runtime: 432.0000 - 29ms/epoch - 29ms/step


				++++++++++++  episode: 279  - step: 4  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 1.0206e-14 - _timestamp: 1654501726.0000 - _runtime: 432.0000 - 36ms/epoch - 36ms/step


				++++++++++++  episode: 279  - step: 5  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 8.0915e-15 - _timestamp: 1654501726.0000 - _runtime: 432.0000 - 41ms/epoch - 41ms/step


				++++++++++++  episo

1/1 - 0s - loss: 7.4940e-15 - _timestamp: 1654501736.0000 - _runtime: 442.0000 - 33ms/epoch - 33ms/step


				++++++++++++  episode: 283  - step: 8  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 8.8480e-15 - _timestamp: 1654501736.0000 - _runtime: 442.0000 - 39ms/epoch - 39ms/step


				++++++++++++  episode: 284  - step: 0  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 7.9587e-15 - _timestamp: 1654501736.0000 - _runtime: 442.0000 - 37ms/epoch - 37ms/step


				++++++++++++  episode: 284  - step: 1  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 7.3880e-15 - _timestamp: 1654501737.0000 - _runtime: 443.0000 - 38ms/epoch - 38ms/step


				++++++++++++  episode: 284  - step: 2  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 9.1854e-15 - _timestamp: 1654501737.0000 - _runtime: 443.0000 - 34ms/epoch - 34ms/step


				++++++++++++  episode: 284  - step: 3  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 7.9441e-



				++++++++++++  episode: 288  - step: 5  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 5.7777e-15 - _timestamp: 1654501746.0000 - _runtime: 452.0000 - 37ms/epoch - 37ms/step


				++++++++++++  episode: 288  - step: 6  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 8.0923e-15 - _timestamp: 1654501746.0000 - _runtime: 452.0000 - 38ms/epoch - 38ms/step


				++++++++++++  episode: 288  - step: 7  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 1.0766e-14 - _timestamp: 1654501747.0000 - _runtime: 453.0000 - 40ms/epoch - 40ms/step


				++++++++++++  episode: 288  - step: 8  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 5.2581e-15 - _timestamp: 1654501747.0000 - _runtime: 453.0000 - 42ms/epoch - 42ms/step


				++++++++++++  episode: 289  - step: 0  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 1.1226e-14 - _timestamp: 1654501747.0000 - _runtime: 453.0000 - 30ms/epoch - 30ms/step


				++++++++++++  episo

1/1 - 0s - loss: 9.2345e-15 - _timestamp: 1654501757.0000 - _runtime: 463.0000 - 42ms/epoch - 42ms/step


				++++++++++++  episode: 293  - step: 3  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 7.1991e-15 - _timestamp: 1654501757.0000 - _runtime: 463.0000 - 31ms/epoch - 31ms/step


				++++++++++++  episode: 293  - step: 4  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 9.6540e-15 - _timestamp: 1654501757.0000 - _runtime: 463.0000 - 37ms/epoch - 37ms/step


				++++++++++++  episode: 293  - step: 5  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 7.3874e-15 - _timestamp: 1654501757.0000 - _runtime: 463.0000 - 27ms/epoch - 27ms/step


				++++++++++++  episode: 293  - step: 6  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 8.7540e-15 - _timestamp: 1654501758.0000 - _runtime: 464.0000 - 35ms/epoch - 35ms/step


				++++++++++++  episode: 293  - step: 7  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 8.9254e-



				++++++++++++  episode: 298  - step: 0  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 8.1629e-15 - _timestamp: 1654501767.0000 - _runtime: 473.0000 - 42ms/epoch - 42ms/step


				++++++++++++  episode: 298  - step: 1  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 8.3351e-15 - _timestamp: 1654501767.0000 - _runtime: 473.0000 - 34ms/epoch - 34ms/step


				++++++++++++  episode: 298  - step: 2  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 8.4919e-15 - _timestamp: 1654501768.0000 - _runtime: 474.0000 - 38ms/epoch - 38ms/step


				++++++++++++  episode: 298  - step: 3  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 6.2521e-15 - _timestamp: 1654501768.0000 - _runtime: 474.0000 - 39ms/epoch - 39ms/step


				++++++++++++  episode: 298  - step: 4  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 7.8235e-15 - _timestamp: 1654501768.0000 - _runtime: 474.0000 - 36ms/epoch - 36ms/step


				++++++++++++  episo

In [14]:
# create a list of all possible states
ensemble = [1, -1]
all_states = [x for x in product(ensemble, repeat=N)]
# compute the true energy minima
trueEnergy = EnergyMinima(states=all_states, inter=environment.interaction, sparseAdj=environment.sp_AdjMat.indices.numpy())
print("\n\nTrue energy minima\n =>", trueEnergy)

# test the trained model
foundEnergy = test(environment, trained_targetModel, L=L)
print("\nFound energy minima\n =>", foundEnergy, "\n\n")

/opt/conda/lib/python3.9/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'states' of function 'EnergyMinima'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../tmp/ipykernel_60488/130955098.py", line 9:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))




True energy minima
 => -9.702928993360777

Found energy minima
 => -4.514650119826711 




In [15]:
wandb.log({
    "True Energy"  : trueEnergy,
    "Found Energy" : foundEnergy
})
wandb.finish()

/opt/conda/lib/python3.9/site-packages/wandb/sdk/lib/ipython.py:47: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML  # type: ignore


Energy,▇█▇▁▃▄▆█▃▅▆▆▂▅▆▃▄▇▁▆▄▃▃▅▄▃▃▇▄█▆▆▅█▇█▇▆▃▅
Episode,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Episode Energy Minima,▆▆▄█▁██▅▇█▇███▃█▆█▁█▅▅▅▅▁▅▅█▅▅█▅▅▅▅▅▅▅▅▇
Found Energy,▁
Step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
True Energy,▁
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▆▆▇▆▆▅▃▆▅▆▅▄▅▃▂▃▃▃▃▂▆▄▃▃▄▅▃▃▃▄▁█▂▂▂▁▃▁▁▂
Energy,-1.61826
Episode,299
Episode Energy Minima,-2.5089
